In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate


In [ ]:
original_signature_dizini = "Downloads/signatures/full_org"
forgery_signature_dizini = "Downloads/signatures/full_forg"


In [ ]:
def load_and_preprocess_data(original_dizini, forgery_dizini, img_size):
    original_signatures = []
    forgery_signatures = []

    for filename in os.listdir(original_dizini):
        if filename.endswith('.DS_Store'):
            continue  # .DS_Store dosyasını görmezden gel
        img_path = os.path.join(original_dizini, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        if img is None:
            print(f"Hata: {img_path} görüntüsü yüklenemedi!")
            continue
        
        img = cv2.resize(img, (img_size, img_size))
        original_signatures.append(img)
    
    for filename in os.listdir(forgery_dizini):
        if filename.endswith('.DS_Store'):
            continue  # .DS_Store dosyasını görmezden gel
        img_path = os.path.join(forgery_dizini, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        if img is None:
            print(f"Hata: {img_path} görüntüsü yüklenemedi!")
            continue
        
        img = cv2.resize(img, (img_size, img_size))
        forgery_signatures.append(img)
    
    original_signatures = np.array(original_signatures, dtype=np.float32) / 255.0
    forgery_signatures = np.array(forgery_signatures, dtype=np.float32) / 255.0
    
    return original_signatures, forgery_signatures


In [ ]:
def find_common_image_size(dizini):
    common_size = None
    
    for filename in os.listdir(dizini):
        if filename.endswith('.DS_Store'):
            continue  # .DS_Store dosyasını görmezden gel
        img_path = os.path.join(dizini, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        if img is None:
            print(f"Hata: {img_path} görüntüsü yüklenemedi!")
            continue
        
        if common_size is None:
            common_size = img.shape
        else:
            common_size = (min(common_size[0], img.shape[0]), min(common_size[1], img.shape[1]))
    
    return common_size


In [ ]:
common_size = find_common_image_size(original_signature_dizini)
if common_size is None:
    common_size = find_common_image_size(forgery_signature_dizini)

# Eğer hiç görüntü yoksa veya boyutları sıfır ise varsayılan boyutu belirleme
if common_size is None or common_size[0] == 0 or common_size[1] == 0:
    img_size = (256, 256)  # Varsayılan boyut
else:
    img_size = common_size


In [ ]:
original_signatures, forgery_signatures = load_and_preprocess_data(original_signature_dizini, forgery_signature_dizini, img_size)


In [ ]:
original_train, original_test = train_test_split(original_signatures, test_size=0.1, random_state=42)
forgery_train, forgery_test = train_test_split(forgery_signatures, test_size=0.1, random_state=42)


In [ ]:
X_train = np.concatenate((original_train, forgery_train), axis=0)
y_train = np.concatenate((np.ones(original_train.shape[0]), np.zeros(forgery_train.shape[0])), axis=0)
X_test = np.concatenate((original_test, forgery_test), axis=0)
y_test = np.concatenate((np.ones(original_test.shape[0]), np.zeros(forgery_test.shape[0])), axis=0)


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model

def unet_model(input_shape):
    inputs = Input(input_shape)

    # Kodlayıcı (Encoder) katmanlar
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)

    # Kod çözücü (Decoder) katmanlar
    up6 = UpSampling2D(size=(2, 2))(conv5)
    up6 = Conv2D(512, 2, activation='relu', padding='same')(up6)
    merge6 = Concatenate(axis=3)([conv4, up6])
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = UpSampling2D(size=(2, 2))(conv6)
    up7 = Conv2D(256, 2, activation='relu', padding='same')(up7)
    merge7 = Concatenate(axis=3)([conv3, up7])
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = UpSampling2D(size=(2, 2))(conv7)
    up8 = Conv2D(128, 2, activation='relu', padding='same')(up8)
    merge8 = Concatenate(axis=3)([conv2, up8])
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = UpSampling2D(size=(2, 2))(conv8)
    up9 = Conv2D(64, 2, activation='relu', padding='same')(up9)
    merge9 = Concatenate(axis=3)([conv1, up9])
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

    # Çıkış katmanı
    outputs = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs, outputs)
    return model

input_shape = (*img_size, 1) 
model = unet_model(input_shape)


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
batch_size = 32
epochs = 10
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))


In [ ]:
print(history.history['accuracy']) 
print(history.history['val_accuracy'])


In [ ]:
test_image_path = "test_fotograf.jpg" 


test_img = cv2.imread(test_image_path, cv2.IMREAD_GRAYSCALE)
test_img = cv2.resize(test_img, (img_size[1], img_size[0]))
test_img = np.array(test_img, dtype=np.float32) / 255.0
test_img = np.expand_dims(test_img, axis=0)  

prediction = model.predict(test_img)

# Tahmin sonucunu yorumlama
threshold = 0.5  # Eşik değeri
if prediction > threshold:
    prediction_text = "Bu gerçek bir imza."
else:
    prediction_text = "Bu sahte bir imza."

    
import matplotlib.pyplot as plt

plt.imshow(test_img[0, :, :, 0], cmap='gray')
plt.title("Photo")
plt.axis("off")
plt.text(10, 10, prediction_text, color='green' if prediction > threshold else 'red')

save_path = "/test_result.jpg/"
plt.savefig(save_path, bbox_inches='tight')

plt.show()
